# Train BeeWorld with TD3 model

### Preparation for colab

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# !pip install gymnasium
# !pip install stable_baselines3, moviepy
# !git clone https://github.com/alTeska/rl-bee-multimodal-sensing.git
# !mv rl-bee-multimodal-sensing/bee.py ./

## Load and setup

In [ ]:
import os

from utils import create_directory, set_device
from train import new_model, load_existing_model, custom_training

DEVICE = set_device()

In [ ]:
# define config as a dict
config = {
    'setup': {
        'path': 'models/bee-world/',
        'gym_name': 'BeeWorld',
        'algorithm': 'TD3',
        'alias': 'round1',
        'continue_training': False,
        'old_alias': ''
    },
    'train': {
        'timesteps': 2000,
        'iter_max': 10,
        'policy_kwargs': {'net_arch': [100, 100], 'activation_fn': 'ReLU'},
        'learning_rate': 0.001
    },
    'test': {
        'prediction_step': 1000
    }
}

In [ ]:

# or Load it from your yml file
# with open("config.yaml", "r") as file:
    # config = yaml.safe_load(file)

## Setup model and environment

In [ ]:
tensorboard_path = os.path.join("/content/drive/", config['setup']['path'], config['setup']['alias'], "logs/")
tensorboard_path

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/models/bee-world/round1/logs/' --port=80

In [ ]:
custom_training(config)

## Train a bit more - NOT ADVISED - better to re run the notebook for your analysis

In [ ]:

config["setup"]["continue_training"] = True
config['setup']['alias'] = 'round2'
config['setup']['old_alias'] = 'round1'

In [ ]:
custom_training(config)